# <font color='black'>Регрессионный анализ социально-экономических процессов, 2024 - 2025 </font>
## <font color='black'> Модели бинарного выбора </font>
В рамках данного практического занятия мы потренируемся в оценивании и интерпретации оценок моделей бинарного выбора. По мотивам статьи [Joan Esteban, Laura Mayoral, Debraj Ray "Ethnicity and Conflict: an Empirical Study"
American Economic Review 2012, 102(4): 1310–1342] рассмотрим зависимость степени конфликта от таких мер, как поляризация и фракционализация. Ниже представлено краткое описание данных:

* prioInt - «Conflict intensity» from PRIO: we assign a value of 0 if there is peace in a given year, a value of 1 if there is a weak conflict in a given year,
and a value of 2 if there is a strong conflict in a given year
* prioIntLag - Lagged conflict intensity
* f - Fractionalization index (data from Fearon (2003b) and the
Ethnologue project)
* p - Polarization index (Group shares are constructed
as above, for f; data on language and linguistic distances come from
Ethnologue)
* gini - Greenberg-Gini index (Ethnologue; Fearon (2003)
* gpd - Log of real GDP per capita
* pop - Log of population
* mount - Percent mountainous terrain
* ncont - Noncontiguous states, referring to countries with territory holding at
least 10,000 people and separated from the land area containing the
capital city either by land or by 100 kilometers of water





In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import statsmodels.formula.api as smf
from scipy.stats import chi2
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc, roc_auc_score

Оставим в массиве только необходимые переменные для анализа. Кроме этого, перекодируем показатель степени конфликта (prioInt) в бинарный
показатель таким образом, чтобы нулю соответствовало отсутствие конфликта,
а единица объединяла бы две категории: «слабый конфликт» и «сильный конфликт». Выполним подобные преобразования применительно и к лагированному
показателю (prioIntLag)

In [ ]:
lab7 = pd.read_stata('lab_logit.dta')
lab7 = lab7[['prioInt', 'prioIntLag', 'f', 'p', 'gini', 'gdp', 'pop', 'mount', 'ncont']].dropna()

In [ ]:
lab7['prioInt_labels'] = lab7['prioInt'].map({0: 'No conflict', 1: 'Weak conflict', 2: 'Strong conflict'})

In [ ]:
distr = sns.countplot(x=lab7['prioInt_labels'], color = 'grey', stat = 'percent', order = lab7['prioInt_labels'].value_counts().index)
percent = lab7['prioInt_labels'].value_counts(ascending=False, normalize=True).values * 100
distr.bar_label(container=distr.containers[0], labels=np.round(percent,2))

In [ ]:
lab7['prioInt_binary'] = lab7['prioInt'].apply(lambda x: 1 if x > 0 else 0)
lab7['prioIntLag_binary'] = lab7['prioIntLag'].apply(lambda x: 1 if x > 0 else 0)

Оценим логит-модель с prioInt_binary в качестве зависимой переменной

In [ ]:
m1_logit = smf.logit("prioInt_binary ~ prioIntLag_binary + f + p + gini + gdp + pop + mount + ncont", data=lab7).fit(cov_type = "HC3")
print(m1_logit.summary())

Сравним полученные оценки с соответствующими оценками probit-модели (данная модель основывается на допущении о стандартном нормальном распределении ошибок)

In [ ]:
m1_probit = smf.probit("prioInt_binary ~ prioIntLag_binary + f + p + gini + gdp + pop + mount + ncont", data=lab7).fit(cov_type = "HC3")
print(m1_probit.summary())

In [ ]:
logit_probit_ratios = pd.DataFrame(
    {"logit": round(m1_logit.params, 3),
     "probit": round(m1_probit.params, 3),
     "logit/probit": round(m1_logit.params/m1_probit.params, 3)}
    )

print(logit_probit_ratios)

Преобразуем оценки логит-модели в отношения шансов:

In [ ]:
odds_ratios = pd.DataFrame(
    {"OR": round(np.exp(m1_logit.params), 3),
     "p-value": round(m1_logit.pvalues, 3),
     "Lower CI": round(np.exp(m1_logit.conf_int()[0]),3),
     "Upper CI": round(np.exp(m1_logit.conf_int()[1]),3)}
    )

print(odds_ratios)

Также возможен вариант интерпретации оценок коэффициентов при непрерывных переменных через предельные эффекты:

In [ ]:
ME = m1_logit.get_margeff()
print(ME.summary())

В качестве предварительной диагностики модели можно использовать тест Хосмера-Лемешева (Hosmer-Lemeshow). Посредством данного теста мы сравним ожидаемые и наблюдаемые частоты по подгруппам (чаще всего берется разделение по децилям). Надо признать, что результаты теста довольно чувствительны к количеству групп, на которые делится массив данных.

In [ ]:
X = lab7[['prioIntLag_binary', 'f', 'p', 'gini', 'gdp', 'pop', 'mount', 'ncont']]
y = lab7[['prioInt_binary']]

In [ ]:
y_prob = m1_logit.predict(X)
y_prob1 = pd.concat([y_prob, y], axis = 1)
y_prob1['decile'] = pd.qcut(y_prob1[0], 10)

In [ ]:
obsevents_pos = y_prob1['prioInt_binary'].groupby(y_prob1.decile, observed = True).sum()
obsevents_neg = y_prob1[0].groupby(y_prob1.decile, observed = True).count() - obsevents_pos
expevents_pos = y_prob1[0].groupby(y_prob1.decile, observed = True).sum()
expevents_neg = y_prob1[0].groupby(y_prob1.decile, observed = True).count() - expevents_pos
decile_dataset = pd.concat([obsevents_pos, obsevents_neg, expevents_pos, expevents_neg], axis = 1)
decile_dataset.columns=['obs_pos','obs_neg','exp_pos', 'exp_neg']
print(decile_dataset)

In [ ]:
hl = ((obsevents_neg - expevents_neg)**2/expevents_neg).sum()+((obsevents_pos - expevents_pos)**2/expevents_pos).sum()
df = 8
pvalue=1-chi2.cdf(hl,df)
print('chi-square: {:.2f}'.format(hl))
print('p-value: {:.2f}'.format(pvalue))

Для лучшего понимания, насколько хорошо модель предсказывает наличие и отсутствие конфликта, представим confusion matrix:

In [ ]:
y_pred = y_prob1[0].apply(lambda x: 1 if x > 0.5 else 0)
confmatrix = confusion_matrix(y_true=lab7['prioInt_binary'], y_pred=y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=confmatrix)
disp.plot()

In [ ]:
TP = confmatrix[1][1]
TN = confmatrix[0][0]
FP = confmatrix[0][1]
FN = confmatrix[1][0]

Accuracy = round((TP + TN) / (TP + TN + FP + FN), 3)
Baseline_Accuracy = round(max((TN + FP), (FN + TP)) / (TP + TN + FP + FN) , 3)

Sensitivity = round(TP / (TP + FN), 3)
Specificity = round(TN / (TN + FP), 3)
ErrorI = round((1 - Specificity), 3)
ErrorII = round((1-Sensitivity), 3)

print('Accuracy: {:.2f}'.format(Accuracy)),
print('Baseline_Accuracy: {:.2f}'.format(Baseline_Accuracy))
print('Sensitivity: {:.2f}'.format(Sensitivity))
print('Specificity: {:.2f}'.format(Specificity))
print('ErrorI: {:.2f}'.format(ErrorI))
print('ErrorII: {:.2f}'.format(ErrorII))

Проследим, как изменяются меры чувствительности и ошибки первого рода в зависимости от выбранного порогового значения:

In [ ]:
fpr, tpr, thresholds = roc_curve(y, y_prob)

fig = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curve (AUC={auc(fpr, tpr):.3f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.show()

In [ ]:
df = pd.DataFrame({
    'Specificity': 1-fpr,
    'Sensitivity': tpr
}, index=thresholds)
df.index.name = "Thresholds"
df.columns.name = "Rate"

fig_thresh = px.line(
    df, title='Sensitivity and Specificity at different thresholds',
    width=700, height=500
)

fig_thresh.update_yaxes(scaleanchor="x", scaleratio=0.75)
fig_thresh.update_xaxes(range=[0, 1], constrain='domain')
fig_thresh.show()

Определим порог, при котором значения специфичности и чувствительности будут сбалансированы:

In [ ]:
diff = np.abs(tpr-(1-fpr))
min_diff = np.argmin(diff)

optimized_threshold = thresholds[min_diff]

optimized_threshold

Переоцените модель при заданном пороговом значении (optimized_threshold) и прокомментируйте, как изменились меры качества модели